# 4.2 머신러닝 모델 평가

- 머신러닝의 목표는 처음 본 데이터에서 잘 동작하는 **일반화**된 모델을 얻는 것
- 이 절에서는 일반화, 즉 머신러닝 모델의 성능을 어떻게 측정하는지에 대해 다룸

### 4.2.1 훈련, 검증, 테스트 세트

- 모델 평가의 핵심은 데이터를 **훈련, 검증, 테스트 3개의 세트**로 나누는 것
- **훈련, 검증 세트**를 이용해 모델을 **훈련, 평가 반복하며 좋은 모델을 얻음**
- **최종적으로 한번 테스트 세트로 얻은 모델을 평가**

- 훈련 세트와 테스트 세트 2개만으로 하지 않는 이유
    - 모델을 개발하며 항상 **하이퍼파라미터(hyperparameter)를 튜닝**하기 때문
    - **튜닝도 결국 어떤 파라미터 공간에서 좋은 설정을 찾는 학습**이므로 테스트 세트로 튜닝하게되면 테스트 세트에 학습이 되는 꼴(즉, 한번 튜닝할 때 마다 모델로 테스트 데이터에 대한 **정보 누설(information leak)**이 일어난다는 의미)
    - 따라서, 검증 세트를 따로 두어 검증 세트에 대해 튜닝 후 테스트 세트로 마지막에 모델의 성능(일반화) 평가
    - **모델은 간접적으로라도 테스트 세트에 대해 어떠한 정보도 얻어서는 안됨**

- 대표적인 3가지 모델 평가 방법
    - 단순 홀드아웃 검증(hold-out validation)
    - K-겹 교차 검증(K-fold cross-validation)
    - 셔플링(shuffling)을 사용한 반복 K-겹 교차 검증(iterated K-fold cross-validation)

#### 단순 홀드아웃 검증

- 데이터의 일정량을 테스트 세트로 떼어 놓음
- 남은 데이터에서 훈련, 검증 분리해서 hyperparameter튜닝
- 마지막에 테스트 세트로 평가
- 데이터가 적을 때는 K-겹 교차 검증이나 반복 K-겹 교차검증을 사용해야 함


- 사이킷 런의 train_test_split()을 사용하면 편리함

In [ ]:
## 홀드아웃 검증 구현 예 ##

# 검증 데이터 수
num_validation_samples = 10000
# 전체 훈련데이터를 섞어줌
np.random.shuffle(data)

# 전체 훈련데이터를 훈련, 검증으로 분리
validation_data = data[:num_validation_samples]
train_data = data[num_validation_samples:]

# 훈련데이터로 학습, 검증 데이터로 평가 반복하며 hyperparameter tuning
model = get_model() # 훈련-평가-튜닝을 반복할 때, 계속 새로운 모델을 만들어야 함
model.train(train_data)
validation_score = model.evaluate(validation_data)

# 전체 훈련 데이터로 학습 후 떼어놓은 마지막으로 테스트셋으로 평가
model = get_model()
# 훈련 + 검증 데이터를 모두 사용
model.train(np.concatenate([train_data, validation_data]))
test_score = model.evaluate(test_data)

#### K-겹 교차 검증
- 데이터의 일정량을 테스트 세트로 떼어 놓음
- 나머지 데이터를 동일한 크기를 가진 K개 분할로 나눔
- 각 분할i에 대해 남은 K-1개의 분할로 모델을 훈련하고 분할 i에서 모델을 평가
- 이때, 각 분할 i가 검증 데이터가 되고, 남은 분할들이 훈련 데이터가 됨
- 얻은 K개의 점수를 평균한 것이 최종 점수이며 이 점수를 지표로 hyperparameter 튜닝
- 마지막에 테스트 세트로 평가
- 모델의 성능이 데이터 분할에 따라 편차가 클 경우에 도움이 됨(주로 데이터의 수가 적은 경우)


- 사이킷런의 cross_validate()를 사용하면 쉽게 구현 가능
- 위 함수를 사용하려면 KerasClassifier나 KerasRegressor클래스로 모델을 감싸야 함

In [ ]:
## K-겹 교차 검증 구현 예

k = 4
num_validation_samples = len(data) // k
np.random.shuffle(data)

# k-겹 교차 검증을 통해 hyperparameter 튜닝
validation_scores = []
for fold in range(k):
    validation_data = data[num_validation_samples * fold:
                           num_validation_samples * (fold+1)]
    train_data = data[:num_validation_samples * fold] + 
                 data[num_validation_samples * (fold+1):]
        
    model = get_model()
    model.train(train_data)
    validation_score = model.evaluate(validation_data)
    validation_scores.append(validation_score)
    
validation_score = np.average(validation_scores)

# 전체 훈련 데이터로 학습 후 떼어놓은 마지막으로 테스트셋으로 평가
model = get_model()
model.train(data)
test_score = model.evaluate(test_data)

#### 셔플링을 사용한 반복 K-겹 교차 검증

- 비교적 가용 데이터가 적고 가능한 정확하게 모델을 평가하고자 할 때 사용
- 캐글과 같은 경연에서 큰 도움이 됨
- K-겹 교차 검증을 여러 번 적용하되 K개의 분할 전에 매번 데이터를 무작위로 섞음
- 최종 점수는 모든 K-겹 교차 검증을 실행해 얻은 점수의 평균

### 4.2.2 모델 평가 방식을 선택할 때 유념해야 할 것

#### 대표성 있는 데이터
- **훈련 세트와 테스트 세트에 모든 클래스의 비율이 고르게** 나누어 져야 함
- 이를 위해 일반적으로 훈련 세트와 테스트 세트로 나누기 전에 데이터를 무작위로 섞는 방법을 사용
- 만약, 특정 클래스의 비율이 현저히 적다면 무작위로 섞기 보다 고르게 나누어지도록 고려해야 함(사이킷 런의 train_test_split()의 stratify 매개변수로 타깃 레이블을 전달해 계층별 분할을 수행할 수 있음)

#### 시간의 방향
- **과거로부터 미래를 예측하는 경우 데이터를 분할하기 전에 무작위로 섞으면 절대 안됨** (예를 들어 날씨, 주식 등)
- 섞게 되면 미래의 정보가 누설되기 때문에 모델이 미래의 데이터로 훈련되는 꼴임
- 훈련 세트에 있는 데이터보다 **테스트 세트에 있는 모든 데이터가 미래의 것이 되도록** 해야함

#### 데이터 중복

- 데이터셋에 데이터 포인트가 2번 등장하면, 데이터를 섞고 훈련, 검증으로 나누었을 때, 훈련, 검증 세트에 데이터 포인트가 중복될 수 있음
- 이럴 경우 훈련 데이터로 테스트하는 최악의 경우가 됨
- **훈련 세트와 검증 세트가 중복되지 않는지 확인**
- 훈련 세트와 데이터 세트에 동일한 데이터가 포함되지 않도록 섞이지 않아야 할 그룹을 지정하여 교차 검증을 하기 위해서는 사이킷 런의 GroupKFold 클래스를 cross_validation()에 적용